In [24]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from collections import Counter
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model ,models, layers, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping,LambdaCallback
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
import pickle
from contextlib import redirect_stdout

In [5]:
def load_data(filename):
    with open(filename, 'rb') as file:
        loaded_data = pickle.load(file)
    return loaded_data

In [6]:
normal_dist = load_data('/home/sm32289/Cat_Pose/AutoEncoder/Model2/Data/2/Normal/dist.pkl')
normal_label = [0] * normal_dist.shape[0]

abnormal_dist = load_data('/home/sm32289/Cat_Pose/AutoEncoder/Model2/Data/2/Abnormal/dist.pkl')
abnormal_label = [1] * abnormal_dist.shape[0]
#y = to_categorical(labels).astype(int)
# Split into train, valid, and test 
x_train, x_test, y_train, y_test = train_test_split(normal_dist, normal_label, test_size=0.1)

x_valid, xx_test, y_valid, yy_test = train_test_split(abnormal_dist, abnormal_label, test_size=0.8)
x_test = np.concatenate([x_test, xx_test])
y_test = np.concatenate([y_test, yy_test])

# CNN Autoencoder

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

class CNNAutoencoder(tf.keras.Model):
    def __init__(self):
        super(CNNAutoencoder, self).__init__()

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.Reshape((40, 15, 15, 1), input_shape=(40, 15, 15)),
            layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
            layers.MaxPooling3D((2, 2, 2), padding='same'),
            layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
            layers.MaxPooling3D((2, 2, 2), padding='same'),
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
            layers.UpSampling3D((2, 2, 2)),
            layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
            layers.UpSampling3D((2, 2, 2)),
            layers.Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same'),
            layers.Cropping3D(((0, 1), (0, 0), (0, 0)))
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

# Create an instance of the CNNAutoencoder model
autoencoder_cnn = CNNAutoencoder()

# Display the model summary
autoencoder_cnn.build((None, 40, 15, 15))
autoencoder_cnn.summary()

autoencoder_cnn.compile(optimizer=Adam(), loss=MeanSquaredError())


# 학습
batch_size = 32
epochs = 50
autoencoder_cnn.fit(x_train, x_train, batch_size=batch_size, epochs=epochs)

# LSTM Autoencoder

In [ ]:
# 데이터 전처리
input_shape = (40, 15, 15)
x_train = x_train.reshape((-1, input_shape[0], input_shape[1] * input_shape[2]))

# 모델 구성
latent_dim = 128  # LSTM 유닛 수 (압축된 표현의 차원)
inputs = Input(shape=(input_shape[0], input_shape[1] * input_shape[2]))
encoded = LSTM(latent_dim, activation='relu')(inputs)

decoded = RepeatVector(input_shape[0])(encoded)
decoded = LSTM(input_shape[1] * input_shape[2], activation='relu', return_sequences=True)(decoded)

autoencoder = Model(inputs, decoded)

# 컴파일
autoencoder.compile(optimizer=Adam(), loss=MeanSquaredError())

# 훈련
batch_size = 32
epochs = 50
autoencoder.fit(x_train, x_train, batch_size=batch_size, epochs=epochs)


# LSTM+CNN autoencoder

In [22]:
from tensorflow.keras import layers, models, Input
class DynamicLSTMCNNAutoEncoder(models.Model):
    def __init__(self):
        super(DynamicLSTMCNNAutoEncoder, self).__init__()

        # Encoder
        self.encoder = models.Sequential([
            layers.Reshape((-1, 1)),  # Flatten the input
            layers.Reshape((40, 15, 15, 1)),  # Reshape according to the input shape
            layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
            layers.MaxPooling3D((2, 2, 2), padding='same'),
            layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
            layers.MaxPooling3D((2, 2, 2), padding='same'),
            layers.Reshape((-1, 64)),  # Flatten the output of Conv3D
            layers.LSTM(128, activation='relu', return_sequences=True)
        ])

        # Decoder
        self.decoder = models.Sequential([
            layers.LSTM(64, activation='relu', return_sequences=True, input_shape=(None, 128)),
            layers.Reshape((-1, 4, 4, 64)),  # Reshape according to the output shape of encoder
            layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
            layers.UpSampling3D((2, 2, 2)),
            layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
            layers.UpSampling3D((2, 2, 2)),
            layers.Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

# Example usage:
model = DynamicLSTMCNNAutoEncoder()
model.build((None, None))  # Build the model with dynamic input size
model.summary()


Model: "dynamic_lstmcnn_auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_18 (Sequential)  (None, 160, 128)          155072    
                                                                 
 sequential_19 (Sequential)  (None, None, 16, 16, 1)   216257    
                                                                 
Total params: 371329 (1.42 MB)
Trainable params: 371329 (1.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
